# This is used for importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# Import and read text from Wikipedia

In [2]:
web_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
web_text= requests.get(web_link).text
Canada_data = BeautifulSoup(web_text, 'xml')

# Coding for import and clean Toronto data frame from Wikipedia

In [4]:
table = Canada_data.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
           
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue    
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

# Merge postal code with neighborhood

In [6]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


# Toronto Neighborhood Table

In [7]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M1E,Scarborough,"Morningside, West Hill"
1,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
2,M2R,North York,Willowdale West
3,M6J,West Toronto,"Little Portugal, Trinity"
4,M4H,East York,Thorncliffe Park
5,M1T,Scarborough,Tam O'Shanter
6,M5R,Central Toronto,"The Annex, Yorkville"
7,M6M,York,"Keelesdale, Mount Dennis, Silverthorn"
8,M8V,Etobicoke,New Toronto
9,M4X,Downtown Toronto,"Cabbagetown, St. James Town"


In [8]:
df_toronto.shape

(77, 3)

# Read csv file and merge the Latitude and Longtitude together

In [14]:
dfll= pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df_toronto.set_index("Postcode")
toronto_data2=pd.merge(df_toronto, dfll)
toronto_data2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
1,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
2,M2R,North York,Willowdale West,43.782736,-79.442259
3,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
4,M4H,East York,Thorncliffe Park,43.705369,-79.349372
